Đề bài: Xây dựng lớp FactorInteger

Mục tiêu: Xây dựng một lớp Python để phân tích thừa số nguyên tố của một số nguyên dương n.

Yêu cầu:

1.	Thuộc tính: 
    - n: Số nguyên dương cần phân tích.

2.	Phương thức phân tích thừa số nguyên tố:
    
    - trial_factorization(): example 3.22
    
    - pollard_rho(): example 4.31
    
    - pollard_pminus1(): section 6.1, example 6.6,6.7.

3.	So sánh running time:
    
    Viết một phương thức compare_methods() để so sánh thời gian chạy của ba phương pháp trên với cùng một số n.

4.	Chạy test cases:

    Lưu ý: Không dùng sympy. Hướng dẫn cách nhập test case.

    - Case 1: n là một số nguyên tố hoặc hợp số < 1000.

    - Case 2: lấy ba số nguyên tố p,q, và r có 5 chữ số rồi nhân lại với nhau ta được n=pqr

    - Case 3: Lên google tìm một số nguyên tố có 7 hoặc 8 chữ số rồi nhập vào.

In [1]:
import random
from time import perf_counter
import math
from collections.abc import Callable
from functools import wraps
import numpy as np
from itertools import combinations

def timing(func: Callable):
    """Output time taken of function"""

    @wraps(func)
    def wrapper(*args, **kwargs):
        tic = perf_counter()
        result = func(*args, **kwargs)
        duration = np.format_float_positional(perf_counter() - tic)
        print(f' - {func.__name__}() took {duration}s')
        return result

    return wrapper

def gcd(a, b):
    """Find greatest common divisor using Euclidean algorithm"""

    while b > 0:
        remainder = a % b
        a = b
        b = remainder 
    return a

In [2]:
class FactorInteger:
    def __init__(self, number) -> None:
        while not isinstance(number, int) or number < 0:
            print(f'{number} is invalid. Number must be integer positive')
            try:
                number = input('Enter number: ')
                number = int(number)
            except ValueError:
                pass

        self.number = number
    
    def __repr__(self):
        return str(self.number)

    @timing
    def trial_factorization(self, get_duration=False):
        if get_duration:
            tic = perf_counter()
        n = self.number
        i = 2
        result = []
        while n != 1:
            if n % i == 0:
                result.append(i)
                n //= i
                if i > int(math.sqrt(n)):
                    result.append(n)
                    if get_duration:
                        return perf_counter() - tic
                    return result
            else:
                i += 1

        if get_duration:
            return perf_counter() - tic        
        return result
    
    @timing
    def pollard_rho(self, k=10, get_duration=False):
        n = self.number
        if get_duration:
            tic = perf_counter()

        result = []
        while n % 2 == 0:
            result.append(2)
            n //= 2

        def f(x):
            return (x**2 + 1) % n
        
        factors = []
        def factorize(n):
            x = 2
            # x = random.randint(2, n-1)
            y = x
            d = 1
            while d == 1:
                x = f(x)
                y = f(f(y))
                d = gcd(abs(x - y), n)
        
            if d == n:
                return None
            else:
                return d
                
        for _ in range(k):
            d = factorize(n)
            if d is not None:
                result.append(d)
                n //= d

        result.append(n)

        if get_duration:
            return perf_counter() - tic
        return result

    @timing
    def pollard_pminus1(self, k=10, get_duration=False):
        n = self.number
        if get_duration:
            tic = perf_counter()

        result = []
        while n % 2 == 0:
            result.append(2)
            n //= 2

        def factorize(k=10):
            r = 2
            for j in range(1, k + 1):
                r = pow(r, j, n)
                d = gcd(r-1, n)
                if d > 1 and d < n:
                    return d

            return None
        
        for _ in range(k):
            d = factorize(k)
            if d is not None:
                result.append(d)
                n //= d

        result.append(n)

        if get_duration:
            return perf_counter() - tic
        return result


    def compare_methods(self) -> None:
        result = {}
        result['trial_factorization'] = self.trial_factorization(get_duration=True)
        result['pollard_rho'] = self.pollard_rho(get_duration=True)
        result['polard_pminus1'] = self.pollard_pminus1(get_duration=True)

        result = {k: v for k, v in 
                  sorted(result.items(), key=lambda item: item[1])}
        print('=> Running time: ', end='')
        for i, (func, duration) in enumerate(result.items()):
            if i == 2:
                print(f'{func}', end='')
            else:
                print(f'{func} > ', end='')
        print()        

In [3]:
print(FactorInteger(8051).trial_factorization())
print(FactorInteger(8051).pollard_rho())

 - trial_factorization() took 0.000007700000423938036s
[83, 97]
 - pollard_rho() took 0.00004710000939667225s
[97, 83]


In [4]:
print(FactorInteger(5157437).trial_factorization())
print(FactorInteger(5157437).pollard_pminus1())

 - trial_factorization() took 0.00009339999814983457s
[2269, 2273]
 - pollard_pminus1() took 0.0000788999896030873s
[2269, 2273]


# Case 1

In [5]:
FactorInteger(42833).trial_factorization()

 - trial_factorization() took 0.000006200003554113209s


[7, 29, 211]

In [11]:
for _ in range(5):
    n = FactorInteger(random.randint(2, 1000))
    print(f'{n = }')
    print(n.trial_factorization())
    print(n.pollard_rho())
    print(n.pollard_pminus1())

    n.compare_methods()
    print()

n = 654
 - trial_factorization() took 0.00001679999695625156s
[2, 3, 109, 1]
 - pollard_rho() took 0.00017210000078193843s
[2, 3, 109]
 - pollard_pminus1() took 0.00006489999941550195s
[2, 3, 109]
 - trial_factorization() took 0.000006999995093792677s
 - pollard_rho() took 0.0001300999865634367s
 - pollard_pminus1() took 0.00006189999112393707s
=> Running time: trial_factorization > polard_pminus1 > pollard_rho

n = 403
 - trial_factorization() took 0.0000014999968698248267s
[13, 31]
 - pollard_rho() took 0.000034500000765547156s
[31, 13]
 - pollard_pminus1() took 0.00008719999459572136s
[13, 31]
 - trial_factorization() took 0.0000026000052457675338s
 - pollard_rho() took 0.000053099996875971556s
 - pollard_pminus1() took 0.00009480000881012529s
=> Running time: trial_factorization > pollard_rho > polard_pminus1

n = 534
 - trial_factorization() took 0.000007700000423938036s
[2, 3, 89, 1]
 - pollard_rho() took 0.00018179998733103275s
[2, 3, 89]
 - pollard_pminus1() took 0.000106699997

# Case 2

In [7]:
def generate_random_prime():
    """Generate a prime have 5 digits"""
    def is_prime(n) -> bool:
        if n == 2:
            return True

        if n < 2 or n % 2 == 0:
            return False

        i = 3
        while i < math.floor(math.sqrt(n)) + 1:
            if n % i == 0:
                return False
            i += 2

        return True
    
    checked = set()
    while True:
        n = random.randint(10001, 99999)
        while n % 2 == 0 or n in checked:
            n = random.randint(1001, 9999)

        if is_prime(n):
            return n
        else:
            checked.add(n)

In [12]:
for _ in range(5):
    p = generate_random_prime()
    q = generate_random_prime()
    r = generate_random_prime()
    print(f'{p = }'), print(f'{q = }'), print(f'{r = }')

    n = FactorInteger(p * q * r)
    print(f'{n = }')
    print(n.trial_factorization())
    print(n.pollard_rho())
    print(n.pollard_pminus1())

    n.compare_methods()
    print()

p = 7681
q = 76423
r = 68501
n = 40210433820563
 - trial_factorization() took 0.0037382000009529293s
[7681, 68501, 76423]
 - pollard_rho() took 0.005817400000523776s
[7681, 68501, 76423]
 - pollard_pminus1() took 0.00011480000102892518s
[7681, 5235051923]
 - trial_factorization() took 0.0035401000059209764s
 - pollard_rho() took 0.005519299986190163s
 - pollard_pminus1() took 0.00011330000415910035s
=> Running time: polard_pminus1 > trial_factorization > pollard_rho

p = 46703
q = 6427
r = 18269
n = 5483626346689
 - trial_factorization() took 0.0008426000131294131s
[6427, 18269, 46703]
 - pollard_rho() took 0.0017974999936996028s
[6427, 18269, 46703]
 - pollard_pminus1() took 0.00018350000027567148s
[5483626346689]
 - trial_factorization() took 0.0007781000022077933s
 - pollard_rho() took 0.0017823999951360747s
 - pollard_pminus1() took 0.00012689999130088836s
=> Running time: polard_pminus1 > trial_factorization > pollard_rho

p = 46171
q = 75767
r = 4157
n = 14542176018649
 - trial_f

# Case 3

Source: [https://t5k.org/curios/index.php?start=7&stop=8](https://t5k.org/curios/index.php?start=7&stop=8)

In [9]:
n_list = [5858581, 7576757, 10234589, 77767777, 99999989]
for n in n_list:
    print(f'{n = }')
    FactorInteger(n).compare_methods()

n = 5858581
 - trial_factorization() took 0.2413900000101421s
 - pollard_rho() took 0.021628000002237968s
 - pollard_pminus1() took 0.00009380000119563192s
=> Running time: polard_pminus1 > pollard_rho > trial_factorization
n = 7576757
 - trial_factorization() took 0.31605409999610856s
 - pollard_rho() took 0.05750349999289028s
 - pollard_pminus1() took 0.00010910000128205866s
=> Running time: polard_pminus1 > pollard_rho > trial_factorization
n = 10234589
 - trial_factorization() took 0.4363939000031678s
 - pollard_rho() took 0.03014689999690745s
 - pollard_pminus1() took 0.00010599999222904444s
=> Running time: polard_pminus1 > pollard_rho > trial_factorization
n = 77767777
 - trial_factorization() took 3.232870899999398s
 - pollard_rho() took 0.11990599999262486s
 - pollard_pminus1() took 0.00010710000060498714s
=> Running time: polard_pminus1 > pollard_rho > trial_factorization
n = 99999989
 - trial_factorization() took 4.069801900011953s
 - pollard_rho() took 0.05527970001276117s
